<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#List-of-Features-to-Create" data-toc-modified-id="List-of-Features-to-Create-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>List of Features to Create</a></span></li></ul></div>

# List of Features to Create

* \# of unique inpatient beneficiaries
* \# of unique outpatient beneficiaries
* Ratio of inpatient claims
* Whether the provider serves both in/out patients
* Ratio of attending physicians serving for different hospitals (75% threshold)
* Ratio of operating physicians serving for different hospitals (75% threshold)
* Ratio of other physicians serving for different hospitals (75% threshold)
* Ratio of inpatients going to different hospitals (75% threshold)
* Ratio of outpatients going to different hospitals (75% threshold)
* Ratio of inpatients that receive both in/out patient service
* Ratio of outpatients that receive both in/out patient service
* Mean claim duration for inpatients
* Mean claim duration for outpatients
* Mean amount of reimbursed claims in dollars for inpatients
* Mean amount of reimbursed claims in dollars for outpatients
* Total (or mean) deductible paid for inpatients
* Total (or mean) deductible paid for outpatients
* Ratio of claims that didn’t have any physician involved
* Ratio of claims that had all physicians involved
* Mean admission duration for inpatients
* Ratio of patients over age 65 (or average age)
* Ratio of inpatients with top 5 frequent chronic disease (from PotentialFraud)
* Ratio of outpatients with top 5 frequent chronic disease (from PotentialFraud)
* The mean number of chronic condition for inpatients
* The mean number of chronic condition for outpatients
* Ratio of inpatient claims with top 5 admtcode (from PotentialFraud)
* Ratio of outpatient claims with top 5 admtcode (from PotentialFraud)
* avg Insurance covered Ratio for inpatients (Reimbursement/(Reimbursement+Deductible) 
* avg Insurance covered Ratio for outpatients
* Avg revenue per day for inpatients
* Avg revenue per day for outpatients
* Inpatient duplicate ratio
* Outpatient duplicate ratio
* All Diag codes are NaN ratio
* All Proc codes are NaN ratio
* The mean number of chronic condition	
